<a href="https://colab.research.google.com/github/MANDARA-M/testrepo/blob/main/AIQ_dataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark import SparkContext,SQLContext
sc=SparkContext.getOrCreate()
sq=SQLContext(sc) 
rdd=sc.textFile('/content/sample_data/lvl.txt')

rdd.count()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


19

In [ ]:
#remove blank lines
srdd=rdd.filter(lambda l:l.strip())
srdd.count()


17

In [ ]:
wrdd=srdd.flatMap(lambda l:l.split(" "))
wrdd.take(10)

['It', 'only', 'grew', 'stronger', 'by', 'the', 'day.', 'But', 'as', 'always']

In [ ]:
#remove stop words
stopwords=["if","me","by","But","did't","been","than","every","Every","in","is","or","the","and","for","a","an","It","to","on","be","but","of","this","that","as","at","into","are","more","so","with","without","was","don't","you","my","we","there","have","do","along","it","I"]
stpwrd=wrdd.filter(lambda w:w not in stopwords)
wrdd.count(),stpwrd.count()

(692, 398)

In [ ]:
stpwrd.take(15)

['only',
 'grew',
 'stronger',
 'day.',
 'always',
 'didn’t',
 'listen',
 'world,',
 'people',
 'around',
 'me.',
 'If',
 'only',
 'could',
 'brave']

In [ ]:
# remove punctuations

import string
puntwrd=stpwrd.map(lambda w:w.translate(str.maketrans('','',string.punctuation)))
puntwrd.take(10)

['only',
 'grew',
 'stronger',
 'day',
 'always',
 'didn’t',
 'listen',
 'world',
 'people',
 'around']

In [ ]:
# i) Word with highest number of occurrence
grdd=puntwrd.groupBy(lambda w:w)
mrdd=grdd.map(lambda x:(len(x[1]),x[0]))

srdd=mrdd.sortByKey(False)
srdd.take(5)

# "love" is the most popular word

[(15, 'love'), (12, 'You'), (11, 'you'), (8, 'life'), (7, 'day')]

In [ ]:
# ii) Character with highest number of occurrence

grdd=puntwrd.groupBy(lambda w:w[0:1])
mrdd=grdd.map(lambda x:(len(x[1]),x[0]))
srdd=mrdd.sortByKey(False)
srdd.take(5)

# l is popular character


[(44, 'l'), (34, 'm'), (32, 'w'), (29, 's'), (27, 'a')]

In [ ]:
# iii) Compute occurrence of any given word accepted from keyboard

word=input("enter word:")
prdd=puntwrd.groupBy(lambda w:w)
krdd=prdd.map(lambda w:(len(w[1]),w[0]))
nrdd=krdd.filter(lambda w:word in w)
nrdd.collect()

enter word:life


[(8, 'life')]

In [ ]:
# iv) Line sorted in descending order of number of words they contain

w2rdd=puntwrd.map(lambda w:(w,1))
w2rdd=w2rdd.reduceByKey(lambda x,y:x+y)
w2rdd.map(lambda x:(x[1],x[0])).sortByKey(False).take(20)

[(15, 'love'),
 (12, 'You'),
 (11, 'you'),
 (8, 'life'),
 (7, 'day'),
 (6, 'world'),
 (6, 'am'),
 (6, 'it’s'),
 (6, 'your'),
 (6, 'because'),
 (4, 'even'),
 (4, 'know'),
 (4, 'anything'),
 (4, 'look'),
 (4, 'like'),
 (4, 'me'),
 (4, 'what'),
 (4, 'much'),
 (4, 'one'),
 (4, 'want')]

In [ ]:
# v)Take 80% sample first time and perform intersection between them and check number of lines common.

samplerdd1=puntwrd.sample(True,0.8,20)
samplerdd2=puntwrd.sample(True,0.8,21)
intersectionsample=samplerdd1.intersection(samplerdd2)
samplerdd1.count(),samplerdd2.count()
print(intersectionsample.count(),"no of lines are common in both the samples")



93 no of lines are common in both the samples


In [ ]:
# v)Take 80% sample twice and perform intersection between them and check number of lines common.

samplerdd1=puntwrd.sample(True,0.8,40)
samplerdd2=puntwrd.sample(True,0.8,41)
intersectionsample=samplerdd1.intersection(samplerdd2)
samplerdd1.count(),samplerdd2.count()
print(intersectionsample.count(),"no of lines are common in both the samples")

#98 lines are common in both the samples

98 no of lines are common in both the samples


QUESTION 2


In [ ]:

from pyspark.sql.session import SparkSession
sess=SparkSession.builder.getOrCreate()
import pyspark.sql.types as typ
labels=[
    ('age',typ.IntegerType()),
    ('gender',typ.StringType()),
    ('miles',typ.IntegerType()),
    ('debt',typ.IntegerType()),
    ('income',typ.IntegerType()),
    ('sales',typ.IntegerType())
]
schema = typ.StructType([
typ.StructField(e[0], e[1], True) for e in labels
])
cars=sess.read.csv('/content/sample_data/cars.csv',header=True,schema=schema)
cars.count()
cars.printSchema()


root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- miles: integer (nullable = true)
 |-- debt: integer (nullable = true)
 |-- income: integer (nullable = true)
 |-- sales: integer (nullable = true)



In [ ]:
# i) Find the number of records whose income > 8000

cars.filter(cars.income>8000).count()


295

In [ ]:
  # ii)Find the number of records whose income <4000 and sales>1000

print(sq.sql('select income,sales,count(*) as Count from car_table where income<4000 and sales>1000 group by income,sales order by count(*) desc').count(),"number of records having income<4000 & sales>1000 ")

sq.sql('select income,sales from car_table where income<4000 and sales>1000').show(3)



233 number of records having income<4000 & sales>1000 
+------+-----+
|income|sales|
+------+-----+
|  2677| 1792|
|  1262| 2102|
|  1376| 3197|
+------+-----+
only showing top 3 rows



In [ ]:
# iii) Find the number of records for different ages and order them in descending order

from  pyspark.sql.functions import col
print("The no of records are",cars.groupBy('Age').count().orderBy(col('count').desc()).count())
cars.groupBy('Age').count().orderBy(col('count').desc()).show(10)

The no of records are 42
+---+-----+
|Age|count|
+---+-----+
| 25|   41|
| 30|   34|
| 27|   33|
| 37|   32|
| 24|   32|
| 36|   30|
| 28|   30|
| 26|   29|
| 29|   29|
| 21|   29|
+---+-----+
only showing top 10 rows



In [ ]:
# iv) Find mean sales of male(1) and female (0)

cars.groupBy('Gender').agg({'Sales':'Mean'}).show()

+------+------------------+
|Gender|        avg(Sales)|
+------+------------------+
|     0|12024.953091684434|
|     1|11371.726720647774|
+------+------------------+



In [ ]:
# v) Find the descriptive statistics of debt

cars.select('Debt').describe().show()

+-------+------------------+
|summary|              Debt|
+-------+------------------+
|  count|               963|
|   mean|14109.004153686397|
| stddev| 18273.70248104874|
|    min|                 0|
|    max|             59770|
+-------+------------------+



In [ ]:
# vi) Find correlation between miles and debt

cars.crosstab('miles','debt').show(10)

+----------+---+---+----+----+-----+-----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+---+----+---+-----+-----+----+-----+----+-----+----+----+----+---+----+-----+----+----+-----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+----+----+----+----+----+----+-----+----+----+----+----+----+-----+-----+---+----+----+----+-----+----+-----+-----+----+----+---+-----+---+-----+----+-----+-----+-----+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+-----+----+----+----+----+----+-----+-----+----+---+-----+----+----+----+----+----+----+----+---+-----+----+----+-----+-----+---+----+----+---+-----+---+----+---+----+-----+----+----+----+----+-----+----+-----+----+-----+----+----+-----+----+----+-----+-----+----+----+----+----+-----+----+---+----+----+-----+-----+-----+----+-----+----+----+----+-----+----+---+---+-----+----+---+-----+-----+-----+----+-----+-----+-----+----+----+-----+---+----+----+----+----

In [ ]:
cars.createOrReplaceTempView('car_table')

In [ ]:
# vii)Convert the dataframe to table and write an SQL query for finding grouping of age,gender and number of records of each group



from pyspark import SparkContext,SQLContext
sc=SparkContext.getOrCreate()
sq=SQLContext(sc) 
sq.sql('select age,gender,count(*) as Count from car_table group by age,gender order by count(*) desc').show()





+---+------+-----+
|age|gender|Count|
+---+------+-----+
| 24|     1|   21|
| 25|     0|   21|
| 36|     0|   20|
| 25|     1|   20|
| 30|     1|   20|
| 37|     1|   20|
| 27|     1|   18|
| 22|     0|   17|
| 21|     0|   17|
| 29|     1|   16|
| 26|     0|   16|
| 28|     1|   16|
| 38|     1|   15|
| 23|     1|   15|
| 20|     0|   15|
| 43|     0|   15|
| 27|     0|   15|
| 40|     1|   14|
| 48|     0|   14|
| 28|     0|   14|
+---+------+-----+
only showing top 20 rows



In [ ]:
# viii) Store the results of question (iii) in a csv called carf.csv

from  pyspark.sql.functions import col
f=cars.groupBy('Age').count().orderBy(col('count').desc())
f.write.format('csv').save('/content/sample_data/carf.csv')

QUESTION 3
Machine Learning

In [ ]:
# i) Data preprocessing

import pyspark.sql.types as typ
labels=[
        ('PM25',typ.DoubleType()),
        ('PM10',typ.DoubleType()),
        ('NO',typ.DoubleType()),
        ('NO2',typ.DoubleType()),
        ('NOx',typ.DoubleType()),
        ('NH3',typ.DoubleType()),
        ('CO',typ.DoubleType()),
        ('SO2',typ.DoubleType()),
        ('O3',typ.DoubleType()),
        ('Benzene',typ.DoubleType()),
        ('Toluene',typ.DoubleType()),
        ('Xylene',typ.DoubleType()),
        ('AQI',typ.DoubleType())
        
          ]
schema = typ.StructType([
typ.StructField(e[0], e[1], False) for e in labels
])
aqd=sess.read.csv('/content/sample_data/airqualitydataset.csv',header=True,schema=schema)
aqd.show(8),aqd.printSchema()



+-----+------+----+-----+-----+-----+----+-----+------+-------+-------+------+-----+
| PM25|  PM10|  NO|  NO2|  NOx|  NH3|  CO|  SO2|    O3|Benzene|Toluene|Xylene|  AQI|
+-----+------+----+-----+-----+-----+----+-----+------+-------+-------+------+-----+
| 81.4| 124.5|1.44| 20.5|12.08|10.72|0.12|15.24|127.09|    0.2|    6.5|  0.06|184.0|
|78.32|129.06|1.26| 26.0|14.85|10.28|0.14|26.96|117.44|   0.22|   7.95|  0.08|197.0|
|88.76|135.32| 6.6|30.85|21.77|12.91|0.11|33.59|111.81|   0.29|   7.63|  0.12|198.0|
|64.18|104.09|2.56|28.07|17.01|11.42|0.09| 19.0|138.18|   0.17|   5.02|  0.07|188.0|
|72.47|114.84|5.23| 23.2|16.59|12.25|0.16|10.55|109.74|   0.21|   4.71|  0.08|173.0|
| 69.8|114.86|4.69|20.17|14.54|10.95|0.12|14.07|118.09|   0.16|   3.52|  0.06|165.0|
|73.96|113.56|4.58|19.29|13.97|10.95| 0.1| 13.9| 123.8|   0.17|   2.85|  0.04|191.0|
| 89.9| 140.2|7.71|26.19|19.87|13.12| 0.1|19.37|128.73|   0.25|   2.79|  0.07|191.0|
+-----+------+----+-----+-----+-----+----+-----+------+-------+--

(None, None)

In [ ]:
#drop duplicates and null values
aqd.count(),aqd.dropDuplicates().count()

(10314, 10314)

In [ ]:
# ii) Filtering of important columns contributing to AQI


import pyspark.mllib.stat as st
num_rdd=aqd.rdd
num_rdd=num_rdd.map(lambda row:[e for e in row])
num_rdd.collect()
mllib_stats=st.Statistics.colStats(num_rdd)

In [ ]:
mllib_stats.mean(),mllib_stats.variance()

(array([ 52.48210006, 108.49481288,  12.20489723,  33.18832946,
         29.91121001,  17.09981191,   0.69924181,   9.90713011,
         32.22449777,   4.42978864,  12.12051677,   2.86233275,
        119.84700407]),
 array([1.85770842e+03, 4.60787302e+03, 3.56483070e+02, 5.24865325e+02,
        7.94565153e+02, 1.62238595e+02, 1.90896526e-01, 6.28251707e+01,
        3.97490429e+02, 1.80077491e+02, 5.65234358e+02, 4.96828884e+01,
        5.63632644e+03]))

In [ ]:
#corellation matrix computation

corrs=st.Statistics.corr(num_rdd)
corrs[-1]

array([0.91519013, 0.90473294, 0.47765931, 0.54618293, 0.58023652,
       0.34522906, 0.60818697, 0.18335878, 0.09487005, 0.14251333,
       0.21557067, 0.14224832, 1.        ])

In [ ]:
aqd.columns

['PM25',
 'PM10',
 'NO',
 'NO2',
 'NOx',
 'NH3',
 'CO',
 'SO2',
 'O3',
 'Benzene',
 'Toluene',
 'Xylene',
 'AQI']

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['PM25',
 'PM10',
 'NO',
 'NO2',
 'NOx',
 'NH3',
 'CO',
'AQI'
], outputCol = 'features')
out = vectorAssembler.transform(aqd)
out = out.select(['features', 'AQI'])
out.show(5)

+--------------------+-----+
|            features|  AQI|
+--------------------+-----+
|[81.4,124.5,1.44,...|184.0|
|[78.32,129.06,1.2...|197.0|
|[88.76,135.32,6.6...|198.0|
|[64.18,104.09,2.5...|188.0|
|[72.47,114.84,5.2...|173.0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# iii)Split into 70, 30 train and test dataframes
splits=out.randomSplit([0.7,0.3])
aqd_train=splits[0]
aqd_test=splits[1]
aqd_train.count(),aqd_test.count()

(7284, 3030)

In [ ]:
# iv) Apply Linear Regression MODEL

from pyspark.ml.regression import LinearRegression
lin_reg= LinearRegression(featuresCol = 'features', labelCol='AQI', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lin_reg_model=lin_reg.fit(aqd_train)
print("Weights: " + str(lin_reg_model.coefficients))
print("Bias: " + str(lin_reg_model.intercept))

Weights: [0.1679454088414538,0.06527431996648803,-0.05003690328102676,-0.09162242893990923,0.058678463859504766,0.005942229546029458,0.0,0.8627245066872985]
Bias: 2.299864039672635


In [ ]:
# Decision tree model

from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'AQI')
dt_model = dt.fit(aqd_train)



In [ ]:
# Gradient Boost Regression Model

from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'AQI', maxIter=10)
gbt_model = gbt.fit(aqd_train)

In [ ]:
 #  v) prediction over test data of  Linear Regression model

lin_reg_predictions = lin_reg_model.transform(aqd_test)
lin_reg_predictions.select("prediction","AQI","features").show()

+------------------+----+--------------------+
|        prediction| AQI|            features|
+------------------+----+--------------------+
|29.797637350911447|30.0|[1.09,23.65,6.69,...|
| 20.69206719888589|21.0|[1.31,10.83,4.44,...|
|  44.2771241525348|47.0|[2.98,17.03,1.17,...|
| 44.78407555686312|50.0|[3.36,11.46,5.04,...|
|21.401777291062032|21.0|[3.51,15.13,3.95,...|
|33.627820050696364|34.0|[3.82,28.98,0.41,...|
| 69.93699425272642|76.0|[3.97,22.27,6.09,...|
| 62.67673894973149|66.0|[4.03,43.13,7.17,...|
| 42.93006306369996|46.0|[4.11,12.83,4.07,...|
| 32.79209759215079|34.0|[4.29,20.42,4.4,1...|
| 33.72056956467804|35.0|[4.3,10.48,1.45,3...|
|  34.2028240878203|34.0|[4.42,28.69,6.9,4...|
| 43.69173961014973|46.0|[4.56,18.25,0.4,4...|
| 23.50488187190711|23.0|[4.62,18.93,4.11,...|
|36.717335411905445|38.0|[4.63,26.2,3.94,1...|
| 43.91730743303005|47.0|[4.7,12.83,1.66,1...|
|45.843376517979635|49.0|[4.75,9.25,2.5,7....|
|25.244610062159573|25.0|[4.76,19.44,4.17,...|
| 65.73962228

In [ ]:
#evaluation of accuracy of linear regression model

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="AQI",metricName="r2")
print(lr_evaluator.evaluate(lin_reg_predictions))

0.997435911375589


In [ ]:
#prediction  on decision model
dt_predictions = dt_model.transform(aqd_test)

In [ ]:
# accuracy of dt
dt_evaluator = RegressionEvaluator(
    labelCol="AQI", predictionCol="prediction", metricName="r2")
r2 = dt_evaluator.evaluate(dt_predictions)
print(r2)

0.9785168540799798


In [ ]:
#predictions of gbt

gbt_predictions = gbt_model.transform(aqd_test)
gbt_predictions.select('prediction', 'AQI', 'features').show(7)

+------------------+----+--------------------+
|        prediction| AQI|            features|
+------------------+----+--------------------+
|29.413782916742477|30.0|[1.09,23.65,6.69,...|
| 30.09350622294566|21.0|[1.31,10.83,4.44,...|
| 47.22450614049645|47.0|[2.98,17.03,1.17,...|
| 49.90203775999403|50.0|[3.36,11.46,5.04,...|
|29.413782916742477|21.0|[3.51,15.13,3.95,...|
|30.472437247620764|34.0|[3.82,28.98,0.41,...|
| 74.54496777349277|76.0|[3.97,22.27,6.09,...|
+------------------+----+--------------------+
only showing top 7 rows



In [ ]:
# accuracy of gbt
gbt_evaluator = RegressionEvaluator(
    labelCol="AQI", predictionCol="prediction", metricName="r2")
r2 = gbt_evaluator.evaluate(gbt_predictions)
print(r2)

0.9809143952534611
